In [152]:
from pyscopus import Scopus
import pandas as pd
import requests
import xmltodict, json


from lxml import etree
import requests
import xmltodict, json

from bs4 import BeautifulSoup
import numpy as np

In [153]:
#print("Ingrese el dato a buscar:")
#busqueda = input()

Prueba Masiva 

In [154]:
 #Declaracion 
listalinks = list()
listatitulos = list()
listabstract = list()
listautor= list()
listakeywords = list()
listafuente = list()
listkey=list()

list_token_palabra_abstract=[]
list_token_palabra=[]

In [155]:
def consumo_scopus(busqueda):
    
    
    key = '0387c7142db3254ed53c09c131ecf8c0'
    scopus = Scopus(key)
    search_df = scopus.search("ALL("+busqueda+")", count=100)
    for i in range(len(search_df['scopus_id'])):
        try:
            listalinks.append("https://www.scopus.com/inward/record.uri?partnerID=HzOxMe3b&scp="+search_df['scopus_id'][i]+"&origin=inward")
            try:
                pub_abs = scopus.retrieve_abstract(search_df['scopus_id'][i])
                listabstract.append(pub_abs['abstract'])
            except:
                listabstract.append("0")

            try:
                listatitulos.append(search_df['title'][i])
            except:
                listatitulos.append("0")

            try:
                list_aux=[]
                for j in range(len(search_df['authors'][i])):
                    aut=scopus.retrieve_author(search_df['authors'][i][j])
                    list_aux.append(aut['name'])

                listautor.append(list_aux)
            except:
                listautor.append("0")

            listakeywords.append("0")
            listafuente.append("scopus")
            listkey.append(busqueda)    
        except:
            print("Sin link No cargar")
        



In [156]:
def consumo_doaj(busqueda):
    #pasar a ingles 
    #funciona con el spacio 
    url = 'https://www.doaj.org/api/v2/search/articles/all('+busqueda+')?pageSize=100'
    headers = {'Content-Type': 'application/json;charset=UTF-8', 'Access-Control-Allow-Origin': '*'}
    r = requests.get(url, headers=headers)

    #Consumo api  de doaj sobre titulo,url,abstract,author(uspto),keywords
    content = r.json()
    #Ingreso al contenido
    json = content['results']

    for i in range(len(json)):
        try:
            listalinks.append(json[i]['bibjson']['link'][0]['url'])
            try:
                listakeywords.append(json[i]['bibjson']['keywords'])
            except:
                listakeywords.append('0')

            try:
                listatitulos.append(json[i]['bibjson']['title'])
            except:
                listatitulos.append('0')
            try:
                listabstract.append(json[i]['bibjson']['abstract'])
            except:
                listabstract.append('0')
            try:
                aux_aut=[]
                for j in range(len(json[i]['bibjson']['author'])):
                    aux_aut.append(json[i]['bibjson']['author'][0]['name'])
                listautor.append(list(set(aux_aut)))
            except:
                listautor.append('0')

            listafuente.append("doaj")
            listkey.append(busqueda)
        except:
            print("Sin link No cargar")
        
        


In [157]:
def consumo_proquest(busqueda):

    import xmltodict, json
    access_token = '8461684a-5a79-407c-9321-93d8d719ddeb'
    headers = {
        'Content-type': 'text/xml',
        'Authorization': 'Bearer {}'.format(access_token)
    }
    data = '<searchRequest><search><query>'+busqueda+'</query><databases><database>medlineprof</database></databases></search><count>100</count></searchRequest>'
    resp = requests.post('https://api-dialog.proquest.com/v1/search', headers=headers, data=data)
    o = xmltodict.parse(resp.text)
    conta = json.dumps(o)
    d = json.loads(conta)
    

    json = d['searchResponse']['result']['doc']
    
    for i in range(len(json)):
            #print("************************************")
            #print(json[i])
            try:
                response2 = requests.get(json[i]['field'][4]['value'], headers=headers)
                o1 = xmltodict.parse(response2.text)
                listalinks.append(o1['Documents']['Literature']['DocInfo']['URL'])
                import re
                try:

                    listatitulos.append(json[i]['field'][1]['value'])
                except:
                    listatitulos.append("0")


                try:
                    nivel5= o1['Documents']['Literature']['Abstract']
                    listabstract.append(nivel5.get('#text'))
                except:
                    listabstract.append("0")
                try:
                    contributor=o1['Documents']['Literature']['Contributors']['Contributor']
                    aux=list()
                    #print(contributor)
                    for i in range(len(contributor)):
                        if(contributor[i]['@ContribRole']=='Author'):
                            aux.append(contributor[i]['LastName'])
                            
                    listautor.append(aux)
                except:
                    listautor.append("0")
                try:
                    keywords=o1['Documents']['Literature']['Subjects']['HeadingTerms']['HeadingTerm']
                    aux=list()
                    for i in range(len(keywords)):
                        nivel9=keywords[i]['Heading']
                        aux.append(nivel9['#text'])
                    listakeywords.append(aux)
                except:
                    listakeywords.append('0')  

                listafuente.append('ProQuest')
                listkey.append(busqueda)
            except:
                print("no cargar")

In [158]:
def consumo_uspto(busqueda):
    #api lenta 
    #funciona con spacio 
    #ingle
    url_uspto = 'https://developer.uspto.gov/ibd-api/v1/application/publications?searchText='+busqueda+'&start=0&rows=100'
    headers_uspto = {'Content-Type': 'application/json;charset=UTF-8', 'Access-Control-Allow-Origin': '*'}
    r2_uspto = requests.get(url_uspto, headers=headers_uspto) 
    #Consumo api  de propiedad intelectual sobre titulo,url,abstract,nombre(uspto)
    content_uspto = r2_uspto.json()
    json_uspto = content_uspto['results']

    for i in range(len(json_uspto)):
        try:
            listalinks.append(json_uspto[i]['filelocationURI'])
            
            try:
                listatitulos.append(json_uspto[i]['inventionTitle'])
            except:
                listatitulos.append('0')

            try:
                listabstract.append(json_uspto[i]['abstractText'][0])
            except:
                listabstract.append("0")
            try:
                listautor.append(json_uspto[i]['inventorNameArrayText'])
            except:
                listautor.append('0')

            listafuente.append("uspto")
            listakeywords.append(0) 
            listkey.append(busqueda)
        except:
            print("Sin link No cargar")
        
        
            

In [159]:
def consumo_ieeexplore(busqueda):
    #https://www.google.co.uk/search?q=%22medicine%22&tbm=nws&gws_rd=ssl
    url_ieeexplore = 'http://ieeexploreapi.ieee.org/api/v1/search/articles?apikey=3dyyvg36jv78dbuu83q3fzzt&format=json&max_records=100&start_record=1&sort_order=asc&sort_field=article_number&querytext='+busqueda
    headers_ieeexplore = {'Content-Type': 'application/json;charset=UTF-8', 'Access-Control-Allow-Origin': '*'}
    r_ieeexplore = requests.get(url_ieeexplore, headers=headers_ieeexplore)

    content_ieeexplore = r_ieeexplore.json()
    json_ieeexplore = content_ieeexplore['articles']
    for i in range(len(json_ieeexplore)):
        try:
            listalinks.append(json_ieeexplore[i]['pdf_url'])
            
            try:
                listabstract.append(json_ieeexplore[i]['abstract'])
            except:
                listabstract.append('0')
            try:
                listatitulos.append(json_ieeexplore[i]['title'])
            except:
                listatitulos.append('0')


            try:
                aux=list()
                for j in range(len(json_ieeexplore[i]['authors']['authors'])):
                    aux.append(json_ieeexplore[i]['authors']['authors'][j]['full_name'])
                listautor.append(aux)
            except:
                listautor.append("0")
            try:
                listakeywords.append(json_ieeexplore[i]['index_terms']['ieee_terms']['terms'])
            except:
                listakeywords.append('0')  
            listafuente.append('ieeexplore')
            listkey.append(busqueda)
        except:
            print("Sin link No cargar")
        
        
    

In [160]:
def consumo_google_noticas(busqueda):
    from GoogleNews import GoogleNews
    from newspaper import Article
    import pandas as pd
    import time
    from random import randint
    googlenews=GoogleNews(start='05/01/2020',end='19/06/2021')
    googlenews.search(busqueda)
    result=googlenews.result()
    df=pd.DataFrame(result)

    for i in range(10):
        try:
            listalinks.append(df['link'].values[i])
            try:
                listatitulos.append(df['title'].values[i])
            except:
                listatitulos.append("0")
            try:
                listabstract.append(df['desc'].values[i])
            except:
                listabstract.append("0")
            try:
                aux=[]
                aux.append(df['media'].values[i])
                listautor.append(aux)
            except:
                listautor.append("0")

            listakeywords.append("0")
            listafuente.append("google")
            listkey.append(busqueda)
        except:
            print("Sin link No cargar")
            
        
    
    #print('google:link',len(listalinks),' title ',len(listatitulos),'abstract ',len(listabstract),' authors ',len(listautor)," keywords ",len(listakeywords),'fuente', len(listafuente))

    


   

In [161]:


def categorizador_titlulo(listatitulos,name):
    
    import pandas as pd
    import random
    import spacy
    from spacy import displacy
    from spacy.lang.en.stop_words import STOP_WORDS
    nlp = spacy.load('en')
 
    for i in range(len(listatitulos)):
        #print("Categoria Titulo:",i)
        doc = nlp(listatitulos[i])
        list_token=[]
        for token in doc:
            if token.pos_=='ADJ' or token.pos_=="NOUN":
                list_token.append(token.text)

        list_token_palabra.append(list_token)


In [162]:

def categorizador_abstract(listabstract,name):  
    import pandas as pd
    import random
    import spacy
    from spacy import displacy
    from spacy.lang.en.stop_words import STOP_WORDS
    nlp = spacy.load('en')
    
    
    for i in range(len(listabstract)):

        doc = nlp(listabstract[i])
        #print("Categoria Abstract:",i)
        list_token=[]
        for token in doc:
            if token.pos_=='ADJ' or token.pos_=="NOUN":
                list_token.append(token.text)

        list_token_palabra_abstract.append(list_token)


In [163]:
def limpieza(diccionario):
    import pandas as pd  
    dataset=pd.DataFrame(diccionario)
    #print(dataset['link'].duplicated().sum())
    #print("Numero de articulos con repetidos",len(dataset))
    dataset=dataset.drop_duplicates(subset=['link'])
    dataset=dataset.drop_duplicates(subset=['abstract'])
    #print("Numero de articulos sin repetidos",len(dataset))
    
    #Trabajar con categorias en minusculas
    dataset['keywords']= dataset.keywords.astype(str).str.lower()
    dataset['authors']= dataset.authors.astype(str).str.lower()
    dataset['categoriatitulo']= dataset.categoriatitulo.astype(str).str.lower()
    dataset['categoriabstract']= dataset.categoriabstract.astype(str).str.lower()
    
    return dataset
    

In [164]:

dataset_palabras = pd.read_csv('palabras.csv')

#print(dataset_palabras['n.rdfs__label'])
list_palabras_error=[]

for i in range(10):
    
    print("Procesamiento:",i)
    print("Palabra:",dataset_palabras['n.rdfs__label'][i])
    busqueda = dataset_palabras['n.rdfs__label'][i]

    
    try:
        consumo_proquest(busqueda)
        print('link',len(listalinks),' title ',len(listatitulos),'abstract ',len(listabstract),' authors ',len(listautor)," keywords ",len(listakeywords),'fuente', len(listafuente))
    except:
        list_palabras_error.append(dataset_palabras['n.rdfs__label'][i])
    try:
        #consumo_scopus(busqueda)
        print('link',len(listalinks),' title ',len(listatitulos),'abstract ',len(listabstract),' authors ',len(listautor)," keywords ",len(listakeywords),'fuente', len(listafuente))
    except:
        list_palabras_error.append(dataset_palabras['n.rdfs__label'][i])

    try:
        consumo_doaj(busqueda)
        print('link',len(listalinks),' title ',len(listatitulos),'abstract ',len(listabstract),' authors ',len(listautor)," keywords ",len(listakeywords),'fuente', len(listafuente))
    except:
        list_palabras_error.append(dataset_palabras['n.rdfs__label'][i])


    try:
        consumo_uspto(busqueda)
        print('link',len(listalinks),' title ',len(listatitulos),'abstract ',len(listabstract),' authors ',len(listautor)," keywords ",len(listakeywords),'fuente', len(listafuente))
    except:
        list_palabras_error.append(dataset_palabras['n.rdfs__label'][i])


    try:
        consumo_ieeexplore(busqueda)
        print('link',len(listalinks),' title ',len(listatitulos),'abstract ',len(listabstract),' authors ',len(listautor)," keywords ",len(listakeywords),'fuente', len(listafuente))
    except:
        list_palabras_error.append(dataset_palabras['n.rdfs__label'][i])
    
    try:
        consumo_google_noticas(busqueda)
        print('link',len(listalinks),' title ',len(listatitulos),'abstract ',len(listabstract),' authors ',len(listautor)," keywords ",len(listakeywords),'fuente', len(listafuente))
    except:
        list_palabras_error.append(dataset_palabras['n.rdfs__label'][i])
    
    

Procesamiento: 0
Palabra: Ullrich congenital muscular dystrophy
link 7  title  7 abstract  7  authors  7  keywords  7 fuente 7
link 7  title  7 abstract  7  authors  7  keywords  7 fuente 7
link 18  title  18 abstract  18  authors  18  keywords  18 fuente 18
link 118  title  118 abstract  118  authors  118  keywords  118 fuente 118
link 120  title  120 abstract  120  authors  120  keywords  120 fuente 120
Sin link No cargar
link 129  title  129 abstract  129  authors  129  keywords  129 fuente 129
Procesamiento: 1
Palabra: obsolete JMP syndrome
link 129  title  129 abstract  129  authors  129  keywords  129 fuente 129
link 129  title  129 abstract  129  authors  129  keywords  129 fuente 129
link 129  title  129 abstract  129  authors  129  keywords  129 fuente 129
Sin link No cargar
Sin link No cargar
Sin link No cargar
Sin link No cargar
Sin link No cargar
Sin link No cargar
Sin link No cargar
Sin link No cargar
link 131  title  131 abstract  131  authors  131  keywords  131 fuente 1

In [165]:
import threading
#categorizador(listatitulos,listabstract)
list_token_palabra_abstract=[]
list_token_palabra=[]

#CREAMOS PROCESOS A EJECUTAR EN PARALELO.        
t = threading.Thread(target = categorizador_titlulo, args =(listatitulos, 'thread1') )
t2 = threading.Thread(target = categorizador_abstract, args =(listabstract, 'thread2') )


#INICIAMOS PROCESOS.
t.start()
t2.start()


dic_general ={'link':listalinks ,'title':listatitulos,'abstract':listabstract,'authors':listautor,"keywords":listakeywords,'fuente':listafuente,'categoriatitulo':list_token_palabra,'categoriabstract':list_token_palabra_abstract,'key':listkey}


In [168]:
print(len(dic_general['link']))
print(len(dic_general['title']))
print(len(dic_general['abstract']))
print(len(dic_general['authors']))
print(len(dic_general['keywords']))
print(len(dic_general['fuente']))
print(len(dic_general['categoriatitulo']))
print(len(dic_general['categoriabstract']))

552
552
552
552
552
552
552
552


In [170]:
dataset=limpieza(dic_general)
dataset.to_csv('ResultadoCategoriaFinal.csv')